# 블록체인프로그래밍 11주차 과제 201810825 홍종현
은행 컨트랙을 프로그램하여 아래 기능을 실행하도록 하세요.

* 입금액을 매개변수로 넣어서 입금하세요. amount가 실제 입금액과 일치하지 않으면 오류.
	function deposit(uint amount)
* 전액 인출 함수를 구현하세요. 인출 금액을 정하지 않아도 되므로 함수의 인자가 없다.
	function widthdrawAll()
* 컨트랙 잔고확인 함수 (this를 이용한 잔고, 상태변수 잔고 (this 잔고가 맞는지 확인하는 용도)
	function getBalance() public view returns(uint, uint)
* 다른 계정으로 계좌이체 함수
	function forwardTo(address payable _receiver) public payable
* fallback()을 입금가능하도록 구현하세요. fallback이 호출되면 이벤트가 발생하여 전송자와 금액을 출력하세요.
이벤트 함수명은 PrintLog(address from, uint amount)
주피터 노트북에서 컴파일, 배포한 후, 아래 기능을 실행하여 그 결과를 출력하세요.
ganache 8335 띄우고, 노드로 하세요.
- 입금 11111 wei, 222 wei를 하고 컨트랙잔고 11333 wei 출력
- 자신의 2번째 계정으로 계좌이체 333 wei하고 컨트랙잔고 11000 wei 출력, 자신의 2번째 계정 잔고 증가분 (+333 wei) 출력
- 전액 인출하고 11000 wei, 자신의 잔고 증가분 출력 (+11000 wei)
- 컨트랙으로 111 송금하고, deposit()이 아니라 fallback으로 입금하세요. 컨트랙잔고 111 wei 출력.

In [7]:
%%writefile src/Q11.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0; //^0.5.0;

contract Q11 {
    address owner;
    uint balance;
    event PrintLog (address from, uint amount);
    constructor() { //constructor() public {
        owner = msg.sender;
        balance = 0;
    }
    fallback() external payable{ // v0.5.0 function() external {
        balance += msg.value;
        emit PrintLog(msg.sender, msg.value);
    }
    receive() external payable{
        
    }
    function forwardTo(address payable _receiver) public payable onlyOwner {
        _receiver.transfer(msg.value);
    }
    function getBalance() public view returns(uint, uint) {
        return (balance, address(this).balance);
    }
    function deposit(uint amount) public payable  {
        require(msg.value == amount);
        balance += amount;
    }
    function withdrawAll() public onlyOwner minBalance {
        balance -= address(this).balance;
        payable(msg.sender).transfer(address(this).balance); 
    }
    modifier onlyOwner {
        require(msg.sender == owner);
        _;
    }
    modifier minBalance {
        require(address(this).balance>101 wei);
        _;
    }
}

Overwriting src/Q11.sol


In [8]:
!solc-windows.exe --optimize --combined-json abi,bin src/Q11.sol > src/Q11.json

In [9]:
%%writefile src/Q11Deploy.js
var Web3 = require('web3');
var _abiBinJson = require('./Q11.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
//var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts); // reading ['src/Q11.sol:Q11']
console.log("- contract name: ", contractName);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!! //SyntaxError: Unexpected token o in JSON at position 1
_bin=_abiBinJson.contracts[contractName].bin;

//console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 2590210}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/Q11Deploy.js


In [10]:
!node src/Q11Deploy.js

- contract name:  [ 'src/Q11.sol:Q11' ]
Deploying the contract from 0x185805bDDC6762ec152b47EF8d351b413825BbE7
hash: 0x583042fa26a097ae7a4ab405c68eca7d488051711172e81c048d9451ee8d2c9a
---> The contract deployed to: 0xC899B43F09d9Cb5985C7677FcD62FFe68F129838


In [11]:
%%writefile src/Q11Use.js
var Web3=require('web3');
var _abiBinJson = require('./Q11.json');      //importing a javascript file

//var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts); // reading ['src/BankV3.sol:BankV3']
//console.log("- contract name: ", contractName); //or console.log(contractName[0]);
_abiArray=_abiBinJson.contracts[contractName].abi; //use just as read from file
//_abiArray=JSON.parse(JSON.stringify(_abi));
//_abiArray=JSON.parse(_abi);      //JSON parsing needed!!
//_bin=_abiBinJson.contracts[contractName].bin;
console.log("- ABI: " + _abiArray);
//console.log("- Bytecode: " + _bin);
var Q11 = new web3.eth.Contract(_abiArray,"0xC899B43F09d9Cb5985C7677FcD62FFe68F129838");
var event = Q11.events.PrintLog(function (error, result) {
  if (!error)
    console.log("Event fired: "+ JSON.stringify(result.returnValues));
});
//console.log(bank.sendTo(0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480, 100,{from:web3.eth.accounts[0],gas:100000}));

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    const B1_Before = await web3.eth.getBalance(accounts[0]);
    const B2_Before = await web3.eth.getBalance(accounts[1]);

    console.log("\n1.Deposit 11111wei, 222wei and Output contract balance 11333wei");
    await Q11.methods.getBalance().call().then(console.log);
    await Q11.methods.deposit(11111).send({from: accounts[0], value:11111});
    await Q11.methods.getBalance().call().then(console.log);
    await Q11.methods.deposit(222).send({from: accounts[0], value:222});
    await Q11.methods.getBalance().call().then(console.log);
    
    console.log("\n2.Transfer 333wei to my 2nd account and Output contract balance, increment 2nd account");
    await Q11.methods.forwardTo(accounts[1]).send({from: accounts[0], value:333});
    await Q11.methods.getBalance().call().then(console.log);
    const B2_After = await web3.eth.getBalance(accounts[1]);
    console.log("Balance2 Before: " + B2_Before);
    console.log("Balance2 After: " + B2_After);
    console.log("B2: " + (B2_After-B2_Before));

    console.log("\n3.WithdrawAll and Output 11000wei, increment of my balance");
    await Q11.methods.withdrawAll().send({from: accounts[0]});
    const B1_After = await web3.eth.getBalance(accounts[0]);
    await Q11.methods.getBalance().call().then(console.log);
    console.log("Balance1 Before: " + B1_Before);
    console.log("Balance1 After: " + B1_After);
    console.log("B1: " + (B1_After-B1_Before));
    
    console.log("\n4.Send 111 to contract and deposit with fallback, not deposit(), Output contract balance 111wei");
    await web3.eth.sendTransaction({from:accounts[0], to: "0xC899B43F09d9Cb5985C7677FcD62FFe68F129838",data: "0x1111",value:111});
    await Q11.methods.getBalance().call().then(console.log);
    process.exit(1);
}
doIt()

Overwriting src/Q11Use.js


In [12]:
!node src/Q11Use.js

- ABI: [object Object],[object Object],[object Object],[object Object],[object Object],[object Object],[object Object],[object Object]

1.Deposit 11111wei, 222wei and Output contract balance 11333wei
Result { '0': '0', '1': '0' }
Result { '0': '11111', '1': '11111' }
Result { '0': '11333', '1': '11333' }

2.Transfer 333wei to my 2nd account and Output contract balance, increment 2nd account
Result { '0': '11333', '1': '11333' }
Balance2 Before: 1000000000000000000000
Balance2 After: 1000000000000000000333
B2: 0

3.WithdrawAll and Output 11000wei, increment of my balance
Result { '0': '0', '1': '0' }
Balance1 Before: 999999530888000000000
Balance1 After: 999999262803999999667
B1: -268084000063488

4.Send 111 to contract and deposit with fallback, not deposit(), Output contract balance 111wei
Event fired: {"0":"0x185805bDDC6762ec152b47EF8d351b413825BbE7","1":"111","from":"0x185805bDDC6762ec152b47EF8d351b413825BbE7","amount":"111"}
Result { '0': '111', '1': '111' }


### 1. 계좌의 증가분및 차익을 계산하는 과정에서 숫자가 너무 커서 0으로 표기되지만 before와 after값을 비교하면 결과로 나와야하는 값만큼의 차이가 있음을 확인할 수 있음

### 2. 전액 인출후에 차익을 보았을때 11000만큼의 차이가 안나는데 이는 인출과정에서 소모되는 gas가 있기 때문에 정확한 금액이 출력되지 않음